In [1]:
import json
import pandas as pd
from IPython.display import display, Javascript

In [12]:
%%html
<script src="http://d3js.org/d3.v3.min.js" charset="utf-8"></script>
<script src="https://d3js.org/d3-color.v1.min.js"></script>
<script src="https://d3js.org/d3-interpolate.v1.min.js"></script>
<script src="https://d3js.org/d3-scale-chromatic.v1.min.js"></script>

In [47]:
wdfw = pd.read_csv('harvey_count50.csv')#.drop('Unnamed: 0', axis=1)
# vdfw = pd.concat([
#     pd.read_csv('judd_count_senti.csv'),
#     pd.read_csv('rose_count_senti.csv')
# ])
vdfw = pd.read_csv('alyssajudd_sentiment20.csv')#.drop('Unnamed: 0', axis=1)

In [48]:
wdfw.head()

,word,positive count,negative count,positive sentiment,negative sentiment,total count
0,HarveyWeinstein,2419,12128,0.488535,-0.331467,14547
1,NyongLupita,39,2238,0.363805,-0.290939,2277
2,Hollywood,19,1190,0.507044,-0.497840,1209
3,harassment,411,1004,0.577800,-0.569883,1415
4,women,26,697,0.432288,-0.491777,723


In [49]:
wdfw['count'] = wdfw['positive count'] + wdfw['negative count']
vdfw['count'] = vdfw['positive count'] + vdfw['negative count']
wdfw['sentiment'] = (wdfw['positive count'] * wdfw['positive sentiment'] + wdfw['negative count'] * wdfw['negative sentiment']) / wdfw['count']
vdfw['sentiment'] = (vdfw['positive count'] * vdfw['positive sentiment'] + vdfw['negative count'] * vdfw['negative sentiment']) / vdfw['count']

In [50]:
wdfw.dtypes

word                   object
positive count          int64
negative count          int64
positive sentiment    float64
negative sentiment    float64
total count             int64
count                   int64
sentiment             float64
dtype: object

In [51]:
wdfw.word = wdfw.word.str.decode('utf-8', errors='ignore')
vdfw.word = vdfw.word.str.decode('utf-8', errors='ignore')

In [52]:
wdfwh = wdfw.sort_values('sentiment').head(20)
vdfwh = vdfw.sort_values('sentiment').head(20)

In [57]:
Javascript('''
window.data = {{h: {},v: {}, htotal: {}, vtotal: {}}}
console.log(window.data)
'''.format(json.dumps(wdfwh.to_dict(orient='records')), json.dumps(vdfwh.to_dict(orient='records')), wdfwh['count'].sum(), vdfwh['count'].sum()))

<IPython.core.display.Javascript object>

In [58]:
%%html
<script>
console.log(cmap(0.5, [-1, 1], [1,0,0], [0,0,1]))
</script>

In [88]:
%%html
<div id='chart'></div>
<script>
var width = 600, height = 600, margin = 10,
    circleRadius = (width / 2) * .8,
    digitRadius = (width / 2) * .9;

var container = d3.select('#chart');
var svg = container.append("svg")
    .attr("width", width + 2*margin)
    .attr("height", height + 2*margin)
    .append("g")
    .attr("transform", "translate(" + width / 2 + "," + height / 2 + ")");

var hw = svg.append("g")
var vt = svg.append("g")

var width = height = 300,
    circleRadius = (width / 2) * .8,
    digitRadius = (width / 2) * .9;



svg.append("circle")
    .attr("r", circleRadius)
    .style("fill", "none")
    .style('linewidth', '3px')
    .style("stroke", "#ddd");

//var cmaph = d3.interpolateRdBu().domain([-1, 1]);//chroma.scale('RdYlBu').domain([-1, 1]);
//var cmapv = d3.interpolateRdBu().domain([1, -1]);//chroma.scale('RdYlBu').domain([1, -1]);

var cmap = function(v, d, c1, c2){
    var percent = (v - d[0]) / (d[1] - d[0]);
    var c = c1.map((c, i) => c*(1-percent) + c2[i] * percent);
    return 'rgba('+c.join(',')+')';
}

// Position text at X=radius, Y=0 and rotate around the origin to get final position
hw.selectAll("text")
    .data(data.h)
    .enter()
    .append("text")
    .attr("x", digitRadius)
    .style('font-size', function(d, i){ return (d.count/5000 + 0.5)+'em'; })
    .text(function(d, i) { return d.word; })
    .attr("transform", function(d, i) { return "rotate(" + -((i+0.5) / data.h.length * 180) + ")"; });

vt.selectAll("text")
    .data(data.v)
    .enter()
    .append("text")
    .attr("x", digitRadius)
    .style('font-size', function(d, i){ return (d.count/50000 + 0.5)+'em'; })
    .text(function(d, i) { return d.word; })
    .attr("transform", function(d, i) { return "rotate(" + ((data.h.length - i-0.5) / data.h.length * 180) + ")"; });

hw.selectAll("circle")
    .data(data.h)
    .enter()
    .append("circle")
    .attr("cx", circleRadius)
    .attr('r', 5)
    .style("fill", function(d){
        return cmap(d.sentiment, [-1, 1], [255,0,0], [0,0,255]); 
    })
    .attr("transform", function(d, i) { return "rotate(" + -((i+0.5) / data.h.length * 180) + ")"; });

vt.selectAll("circle")
    .data(data.v)
    .enter()
    .append("circle")
    .style("fill", function(d){ 
        return cmap(d.sentiment, [-1, 1], [255,0,0], [0,0,255]); 
    })
    .attr("cx", circleRadius)
    .attr('r', 5)
    .attr("transform", function(d, i) { return "rotate(" + ((data.h.length - i+0.5) / data.h.length * 180) + ")"; });

svg.append('text').attr("y", -digitRadius - 60).text('Weinstein').style('text-anchor', 'middle')
svg.append('text').attr("y", digitRadius + 60).text('Victims').style('text-anchor', 'middle')

</script>